# New Envs

In [3]:
import gym
import numpy as np

from src.algorithm.backward_feature_selection import BackwardFeatureSelector
from src.algorithm.info_theory.entropy import NNEntropyEstimator
from src.algorithm.info_theory.combo_estimators import FastNNEntropyEstimator, NpeetEstimator
from src.wenvs import WrapperEnv
from src.algorithm.utils import episodes_with_len

In [4]:
fest = FastNNEntropyEstimator()
est = NNEntropyEstimator()
nest = NpeetEstimator()

## Lunar Lander

In [3]:
env = gym.make('LunarLander-v2')
wenv = WrapperEnv(env, continuous_state=True, continuous_actions=False)

In [30]:
np.random.seed(0)
wenv.seed(0)

k = 20
num_ep = 1000
trajectories = episodes_with_len(wenv, num_ep, 100, policy=None)

In [31]:
fs = BackwardFeatureSelector(fest, trajectories, nproc=None)

In [32]:
for S, err in fs.try_remove_all(k, 0.9):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7, 8} 0.0


{0, 1, 2, 3, 4, 5, 8} 0.0


{1, 2, 3, 4, 5, 8} 0.26865378998741135


{1, 2, 3, 5, 8} 0.4112143859485764


{8, 2, 3, 5} 0.7516842874208994


{8, 2, 3} 6.298581907545746


{2, 3} 48.5424800235494


{2} 114.66591132627951


set() 118.24830898943405



In [35]:
fss = BackwardFeatureSelector(est, trajectories, nproc=None)
for S, err in fss.try_remove_all(k, 0.9):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7, 8} 0.0


{0, 1, 2, 3, 4, 5, 8} 0.0


{1, 2, 3, 4, 5, 8} 0.26801195410136297


{1, 2, 3, 5, 8} 0.4102295364380056


{8, 2, 3, 5} 0.7524444487934551


{8, 2, 3} 6.282112701454455


{2, 3} 48.170503959648975


{2} 115.15398330801798


set() 117.3385132758146



In [33]:
for S, err in fs.try_remove_all(k, 0.9, sampling="decaying", freq=10):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7, 8} 0.0


{0, 1, 2, 3, 4, 5, 8} 0.0


{1, 2, 3, 4, 5, 8} 0.8372395852100917


{1, 2, 3, 5, 8} 1.2815178137343082


{8, 2, 3, 5} 2.3418297725727184


{8, 2, 3} 19.625539339683357


{2, 3} 150.48219890880927


{2} 351.0122532733329


set() 373.8402341173091



In [34]:
for S, err in fs.try_remove_all(1, 0.9, on_mu=False):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7, 8} 13.616399334589758


{0, 1, 2, 3, 4, 5, 8} 19.25023024534357


{1, 2, 3, 4, 5, 8} 138.6625101897366


{1, 2, 3, 4, 5} 232.34602185597717


{2, 3, 4, 5} 354.6524087564211


{2, 3, 4} 504.2189250826921


{2, 3} 745.8934410843154


{2} 1229.6939122857575


set() 1299.4996276081959



#### With Better policy

In [10]:
from stable_baselines.common import set_global_seeds
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

In [27]:
vec_env = DummyVecEnv([lambda: wenv])  # The algorithms require a vectorized environment to run
model = PPO2(MlpPolicy, vec_env)
hist = model.learn(total_timesteps=50000)
policy = lambda obs: model.predict(obs)[0]

In [29]:
hist = wenv.run_episode(policy=policy, render=True, iterMax=2000)
wenv.close()

In [14]:
np.random.seed(0)
wenv.seed(0)

k = 20
num_ep = 1000
trajectories = episodes_with_len(wenv, num_ep, 100, policy=policy)

In [15]:
fs = BackwardFeatureSelector(fest, trajectories, nproc=None)
for S, err in fs.try_remove_all(k, 0.9):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7, 8} 0.0


{0, 1, 2, 3, 4, 5, 8} 0.0


{1, 2, 3, 4, 5, 8} 0.0537670430967746


{1, 2, 3, 5, 8} 0.0822854425541177


{8, 2, 3, 5} 0.1540220280807119


{8, 2, 3} 1.2578858438225597


{2, 3} 7.715777386314271


{2} 31.21462616496514


set() 14.908703351941027



In [16]:
for S, err in fs.try_remove_all(k, 0.9, sampling="variance"):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7, 8} 0.0


{0, 1, 2, 3, 4, 5, 8} 0.0


{1, 2, 3, 4, 5, 8} 0.14636209048071372


{1, 2, 3, 5, 8} 0.2240200641900595


{8, 2, 3, 5} 0.40918176355606434


{8, 2, 3} 3.42331074823585


{2, 3} 20.502662699767658


{2} 61.159072987955945


set() 65.93688760167481



In [17]:
for S, err in fs.try_remove_all(1, 0.9, on_mu=False):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7, 8} 0.0


{0, 1, 2, 3, 4, 5, 8} 0.0


{0, 1, 2, 3, 4, 5} 10.288543927121909


{1, 2, 3, 4, 5} 16.65024842802595


{1, 2, 4, 5} 25.71421300930861


{1, 2, 4} 36.45204376534203


{1, 2} 54.039000831518365


{2} 86.94400319154605


set() 87.65877762016854



### Continuous version

In [38]:
env = gym.make('LunarLanderContinuous-v2')
wenv = WrapperEnv(env, continuous_state=True, continuous_actions=True)

In [39]:
np.random.seed(0)
wenv.seed(0)

k = 20
num_ep = 1000
trajectories = episodes_with_len(wenv, num_ep, 100, policy=None)

In [40]:
fs = BackwardFeatureSelector(fest, trajectories, nproc=None)

In [41]:
for S, err in fs.try_remove_all(k, 0.9):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7, 8, 9} 0.0


{0, 1, 2, 3, 4, 5, 8, 9} 0.0


{1, 2, 3, 4, 5, 8, 9} 0.17123742000934555


{1, 2, 3, 5, 8, 9} 0.2620969371099906


{2, 3, 5, 8, 9} 0.45877721352141265


{8, 9, 2, 3} 3.99263826842208


{8, 9, 2} 36.16571642060371


{8, 9} 65.69522989579765


{9} 117.82770024239373


set() 121.75479272735656



### Half-Cheetah

In [5]:
env = gym.make('HalfCheetah-v2')
wenv = WrapperEnv(env, continuous_state=True, continuous_actions=True)

In [10]:
np.random.seed(0)
wenv.seed(0)

k = 20
num_ep = 1000
trajectories = episodes_with_len(wenv, num_ep, 100, policy=None)

In [18]:
fs = BackwardFeatureSelector(fest, trajectories, discrete=True, nproc=None)

In [19]:
for S, err in fs.try_remove_all(k, 0.9):
    print(S, err)

{0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22} 0.25892234635183786


{0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22} 0.37281932117631905


{0, 1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22} 0.4628720599386485


{1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22} 0.5466534231587483


{1, 3, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22} 0.617362913630795


...

{17, 20, 21, 22} 3.8124015246589824


{17, 20, 22} 5.067517694091373


{17, 22} 7.265122268298775


{22} 12.728199399177887


set() 6.765146929496406

